
<img src="https://github.com/QuentinAmbard/databricks-demo/raw/main/retail/resources/images/delta-lake-perf-bench.png" width="500" style="float: right; margin-left: 50px"/>

# Delta Lake: Performance made simple

## Blazing fast query at scale

Delta Lake saves all your table metadata in an efficient format, ranging from efficient queries on small tables (GB) to massive PB-scale tables. 

Delta Lake is designed to be smart and do all the hard job for you. It'll automatically tune your table and read the minimum data required to be able to satisfied your query.

This result in **fast read query**, even with a growing number of data/partitions!


In this notebook, we'll see how we can leverage Delta Lake unique capabilities to speedup requests and simplify maintenance operation. For more details, we recommend to read the [documentation](https://docs.databricks.com/delta/file-mgmt.html).

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-engineering&org_id=7474654300667338&notebook=%2F02-Delta-Lake-Performance&demo_name=delta-lake&event=VIEW&path=%2F_dbdemos%2Fdata-engineering%2Fdelta-lake%2F02-Delta-Lake-Performance&version=1">
<!-- [metadata={"description":"Quick introduction to Delta Lake. <br/><i>Use this content for quick Delta demo.</i>",
 "authors":["quentin.ambard@databricks.com"],
 "db_resources":{}}] -->

In [0]:
%run ./_resources/00-setup $reset_all_data=false

## Configuration file

Please change your catalog and schema here to run the demo on a different catalog.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-engineering&org_id=7474654300667338&notebook=%2Fconfig&demo_name=delta-lake&event=VIEW&path=%2F_dbdemos%2Fdata-engineering%2Fdelta-lake%2Fconfig&version=1">


# Technical Setup notebook. Hide this cell results
Initialize dataset to the current user and cleanup data when reset_all_data is set to true

Do not edit

USE CATALOG `main`
using catalog.database `main`.`dbdemos_delta_lake`


##![](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake Liquid Clustering

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/delta/delta-liquid-1.png?raw=true" style="float: right" width="450px">

Data Layout is key to increase performance and query speed. Manual tuning trough hive-style partitioning is not efficient (creating too big or small partitions) and hard to maintain.

To solve this issue, Delta Lake released Liquid Clustering. Liquid will automatically adjusts the data layout based on clustering keys, which helps to avoid the over or under-partitioning problems that can occur with Hive partitioning.

Liquid clustering can be specified on any columns to provide fast access, including high cardinality or data skew. 

* **Liquid is simple**: You set Liquid clustering keys on the columns that are most often queried - no more worrying about traditional considerations like column cardinality, partition ordering, or creating artificial columns that act as perfect partitioning keys.
* **Liquid is efficient**: It incrementally clusters new data, so you don't need to trade off between improving performance with reducing cost/write amplification.
* **Liquid is flexible**: You can quickly change which columns are clustered by Liquid without rewriting existing data.

**Delta Liquid Clustering requires DBR 13.2**

For more details, [please read the documentation](https://docs.databricks.com/delta/clustering.html)

In [0]:
-- Liquid will properly layout the data to speedup queries by firstname or lastname.
-- this is done by adding the CLUSTER BY keyword during your standard table creation. Clustered table can't have partitions.
CREATE OR REPLACE TABLE user_clustering CLUSTER BY (firstname, lastname)
  AS SELECT * FROM user_delta;

num_affected_rows,num_inserted_rows


In [0]:
-- review the table definition, Liquid Clustering appears under "Clustering Information"
DESCRIBE TABLE user_clustering;

col_name,data_type,comment
id,bigint,null
creation_date,timestamp,null
firstname,string,null
lastname,string,null
email,string,null
address,string,null
gender,int,null
age_group,int,null
# Clustering Information,,
# col_name,data_type,comment



### How to trigger liquid clustering
<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/delta/delta-liquid-2.png?raw=true" style="float: right" width="400px">


Liquid clustering is incremental, meaning that data is only rewritten as necessary to accommodate data that needs to be clustered.

For best performance, Databricks recommends scheduling regular OPTIMIZE jobs to cluster data. 

For tables experiencing many updates or inserts, Databricks recommends scheduling an OPTIMIZE job every one or two hours. 

Because liquid clustering is incremental, most OPTIMIZE jobs for clustered tables run quickly. No need to specify any ZORDER columns.

*Note: Liquid clustering will automatically re-arrange your data during writes above a given threshold. As with all indexes, this will add a small write cost.*

In [0]:
OPTIMIZE user_clustering;

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 0, false, 0, 0, 1770101530132, 1770101534795, 8, 0, null, List(0, 0), null, 8, 8, 0, 0, List(145007, true, false, false, null, null, null, null, 0, 0, 0, 0, 1, 145007, 145007, null, log, 16777216, 67108864, 4, 0, 0, null, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(111, 59, 0, 0, 0, 2120), 2, 1, 5, sizeAware))"
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1770101534844, 1770101536844, 8, 0, null, List(0, 0), null, 8, 8, 0, 0, List(145007, false, false, false, null, null, null, post-optimize-compaction, 0, 0, 0, 0, 0, 0, 0, null, null, 33554432, 67108864, 0, 0, 0, null, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, List(0, 0, 848, 0, 0, 0), 15, 1, 1, null))"


In [0]:
VACUUM user_clustering;

path
""


In [0]:
SELECT * FROM user_clustering where firstname = 'Teresa'

id,creation_date,firstname,lastname,email,address,gender,age_group
17179869579,2026-02-03T04:51:37.000Z,Teresa,Dudley,smithedward@sutton.com,"196 Hall Alley North Jennifer, NJ 63776",1,7
60129542796,2026-02-03T04:47:21.000Z,Teresa,Bonilla,derekfritz@rodriguez.com,"95592 Everett Mountain Davechester, CO 67899",0,6
42949673355,2026-02-03T04:51:37.000Z,Teresa,Dudley,smithedward@sutton.com,"196 Hall Alley North Jennifer, NJ 63776",1,3
60129542539,2026-02-03T04:51:37.000Z,Teresa,Dudley,smithedward@sutton.com,"196 Hall Alley North Jennifer, NJ 63776",1,2
8589934987,2026-02-03T04:51:37.000Z,Teresa,Dudley,smithedward@sutton.com,"196 Hall Alley North Jennifer, NJ 63776",0,8
25769804428,2026-02-03T04:47:21.000Z,Teresa,Bonilla,derekfritz@rodriguez.com,"95592 Everett Mountain Davechester, CO 67899",1,8
34359738763,2026-02-03T04:51:37.000Z,Teresa,Dudley,smithedward@sutton.com,"196 Hall Alley North Jennifer, NJ 63776",1,6
34359739020,2026-02-03T04:47:21.000Z,Teresa,Bonilla,derekfritz@rodriguez.com,"95592 Everett Mountain Davechester, CO 67899",1,4
51539608204,2026-02-03T04:47:21.000Z,Teresa,Bonilla,derekfritz@rodriguez.com,"95592 Everett Mountain Davechester, CO 67899",1,7
395,2026-02-03T04:51:37.000Z,Teresa,Dudley,smithedward@sutton.com,"196 Hall Alley North Jennifer, NJ 63776",1,5


### Dynamically changing your clustering columns

Liquid table are flexible, you can change your clustering columns without having to re-write all your data. 

Let's make sure our table provides fast queries for ID:

In [0]:
ALTER TABLE user_clustering CLUSTER BY (id, firstname, lastname);

In [0]:
-- Disable liquid clustering:
ALTER TABLE user_clustering CLUSTER BY NONE;
-- Note: this does not rewrite data that has already been clustered, but prevents future OPTIMIZE operations from using clustering keys.

### Cluster by Auto

In Databricks Runtime 15.4 LTS and above, you can enable automatic liquid clustering for Unity Catalog managed Delta tables. With automatic liquid clustering enabled, Databricks intelligently chooses clustering keys to optimize query performance. You enable automatic liquid clustering using the CLUSTER BY AUTO clause.

When enabled, automatic key selection and clustering operations run asynchronously as a maintenance operation and require that predictive optimization is enabled for the table.

In [0]:
ALTER TABLE user_clustering CLUSTER BY AUTO;

In [0]:
DESCRIBE EXTENDED user_clustering

--Predictive Optimization	ENABLE (inherited from METASTORE unity-catalog-demo)

col_name,data_type,comment
id,bigint,null
creation_date,timestamp,null
firstname,string,null
lastname,string,null
email,string,null
address,string,null
gender,int,null
age_group,int,null
# Clustering Information,,
# col_name,data_type,comment


In [0]:
OPTIMIZE user_clustering;

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1770102359274, 1770102359815, 8, 0, null, List(0, 0), null, 8, 8, 0, 0, null)"


In [0]:
VACUUM user_clustering;

path
""


### Auto Liquid Clustering will dynamically change the cluster keys based on the read/write pattern on the table based on column filters, merge keys etc

###![](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Compacting without Liquid Clustering

While recommended to accelerate your queries, some tables might not always have Liquid Clustering enabled.

Adding data to the table results in new file creation, and your table can quickly have way too many small files which is going to impact performances over time.

This becomes expecially true with streaming operation where you add new data every few seconds, in near realtime.

Just like for Liquid Clusteing, Delta Lake solves this operation with the `OPTIMIZE` command, which is going to optimize the file layout for you, picking the proper file size based on heuristics. As no Cluster are defined, this will simply compact the files.

In [0]:
-- let's compact our table. Note that the engine decided to compact 8 files into 1 ("numFilesAdded": 1, "numFilesRemoved": 8)
OPTIMIZE user_delta 

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1770102394025, 1770102394352, 8, 0, null, List(0, 0), null, 8, 8, 0, 0, null)"


These maintenance operation have to be triggered frequently to keep our table properly optimized.

Using Databricks, you can have your table automatically optimized out of the box, without having you to worry about it. All you have to do is set the [proper table properties](https://docs.databricks.com/optimizations/auto-optimize.html), and the engine will optimize your table when needed, without having you to run manual OPTIMIZE operation.

We strongly recommend to enable this option for all your tables.

In [0]:
ALTER TABLE user_delta SET TBLPROPERTIES (delta.autoOptimize.optimizeWrite = true, delta.autoOptimize.autoCompact = true);

### Note: Auto Optimize with Liquid Clustering

Liquid Clustering will automatically kick off eager optimization starting from a given write size, based on heuristic. 
You can also turn on `delta.autoOptimize.optimizeWrite = true` on your liquid table starting from DBR 13.3 to make sure all writes will be optimized. While you can enable `delta.autoOptimize.autoCompact = true`, it won't have any effect for now (as of DBR 13.3, this might change in the future).

## Legacy file layout optimizations

Liquid Clustering is the future of Delta Lake optimization and query speedup, and we now recommend starting with Liquid Clustering.

Below are previous Delta Lake optimization leveraging Zordering and Partitioning techniques. 

### ZORDER


ZORDER will optimize the file layout by multiple columns, but it's often used in addition to partitioning and is not as efficient as Liquid Clustering. It'll increase the write amplification and won't solve your small partitions issues.

Below are a few examples on how you can leverage ZORDER, but we strongly recommend switching to Liquid Tables instead.

### ![](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Adding indexes (ZORDER) to your table

If you request your table using a specific predicat (ex: username), you can speedup your request by adding an index on these columns. We call this operation ZORDER.

You can ZORDER on any column, especially the one having high cardinality (id, firstname etc). 

*Note: We recommand to stay below 4 ZORDER columns for better query performance.*

In [0]:
OPTIMIZE user_delta ZORDER BY (id, firstname);

-- our next queries using a filter on id or firstname will be much faster
SELECT * FROM user_delta where id = 4 or firstname = 'Quentin';

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake Generated columns for dynamic partitions

Adding partitions to your table is a way of saving data having the same column under the same location. Our engine will then be able to read less data and have better read performances.

Using Delta Lake, partitions can be generated based on expression, and the engine will push-down your predicate applying the same expression even if the request is on the original field.

A typical use-case is to partition per a given time (ex: year, month or even day). 

Our user table has a `creation_date` field. We'll generate a `creation_day` field based on an expression and use it as partition for our table with `GENERATED ALWAYS`.

In addition, we'll let the engine generate incremental ID.

*Note: Remember that partition will also create more files under the hood. You have to be careful using them. Make sure you don't over-partition your table (aim for 100's of partition max, having at least 1GB of data). We don't recommend creating partition on table smaller than 1TB. Use LIQUID CLUSTERING instead.*

In [0]:
CREATE TABLE IF NOT EXISTS user_delta_partition (
  id BIGINT GENERATED ALWAYS AS IDENTITY ( START WITH 10000 INCREMENT BY 1 ), 
  firstname STRING, 
  lastname STRING, 
  email STRING, 
  address STRING, 
  gender INT, 
  age_group INT,
  creation_date timestamp, 
  creation_day date GENERATED ALWAYS AS ( CAST(creation_date AS DATE) ) )
PARTITIONED BY (creation_day);

In [0]:
-- Note that we don't insert data for the creation_day field or id. The engine will handle that for us:
INSERT INTO user_delta_partition (firstname, lastname, email, address, gender, age_group, creation_date) SELECT
  firstname,
  lastname,
  email,
  address,
  gender,
  age_group,
  creation_date
FROM user_delta;

In [0]:
SELECT * FROM user_delta_partition where creation_day = CAST(NOW() as DATE) ;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7125726514234278>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'SELECT * FROM user_delta_partition where creation_day = CAST(NOW() as DATE) ;\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:152, in SqlMagic.sql(self, line, cell)
    148     raise Exception(
    14

That's it! You know how to have super fast queries on top of your Delta Lake tables!


Next: Discover how Delta Lake is an Universal Format with [the 03-Delta-Lake-Uniform notebook]($./03-Delta-Lake-Uniform) or go back to [00-Delta-Lake-Introduction]($./00-Delta-Lake-Introduction).
